In [ ]:
!pip install keras_tuner -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from keras.metrics import categorical_accuracy
import keras_tuner as kt
import tensorflow as tf
import glob
import os


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# merge the files
disease_path = "/content/drive/My Drive/Colab Notebooks/Disease_symptom_and_patient_profile_dataset.csv"

# Concatinate the files
disease_df = pd.read_csv(disease_path,encoding="utf-8")

# Review the DataFrame
disease_df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
344,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
346,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
347,Stroke,Yes,No,Yes,No,90,Female,High,High,Positive


In [ ]:
#identify the data types in the file before transforming
disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood Pressure        349 non-null    object
 8   Cholesterol Level     349 non-null    object
 9   Outcome Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB


In [ ]:
disease_df.nunique()

Disease                 116
Fever                     2
Cough                     2
Fatigue                   2
Difficulty Breathing      2
Age                      26
Gender                    2
Blood Pressure            3
Cholesterol Level         3
Outcome Variable          2
dtype: int64

In [ ]:
#initiate scaling
disease_data_scaled = StandardScaler().fit_transform(disease_df[["Age"]])
#create df with scaled data
disease_scaled_df = pd.DataFrame(disease_data_scaled,columns=["Age"])
#show the newly scaled df
disease_scaled_df.head()

,Age
0,-2.091160
1,-1.631964
2,-1.631964
3,-1.631964
4,-1.631964


In [ ]:
label_encoder = preprocessing.LabelEncoder()

disease_df["Disease"]=label_encoder.fit_transform(disease_df["Disease"])
disease_df['Gender'] = label_encoder.fit_transform(disease_df['Gender'])
disease_df['Difficulty Breathing'] = label_encoder.fit_transform(disease_df['Difficulty Breathing'])
disease_df['Fever'] = label_encoder.fit_transform(disease_df['Fever'])
disease_df['Cough'] = label_encoder.fit_transform(disease_df['Cough'])
disease_df['Fatigue'] = label_encoder.fit_transform(disease_df['Fatigue'])
disease_df['Blood Pressure'] = label_encoder.fit_transform(disease_df['Blood Pressure'])
disease_df['Outcome Variable'] = label_encoder.fit_transform(disease_df['Outcome Variable'])
disease_df['Cholesterol Level'] = label_encoder.fit_transform(disease_df['Cholesterol Level'])

# hot_df = label_encoder.fit_transform(disease_df[['Gender','Difficulty Breathing','Fever','Cough','Fatigue','Blood Pressure','Cholesterol Level']])


In [ ]:
disease_df["Disease"].unique()

array([ 56,  24,  37,   6,  53,   1,   4,  32,  42,  81,  92,  31,  60,
       101, 112,  30,  48,  57,  69,  72,  98, 111,   9,  13,  15,  23,
        52,  70,  73, 113,  77,  84,   7,  21,  39,  51,  74,  90,  93,
        22,  25,  61,  65, 100,  58,  75,  59,   0,  11,  28,  43,  91,
        16,  26,  35,  41,  46,  54,  64, 108,  62,  55,   8,  27,  50,
        68,  79, 109, 115,  14,  83,  95,  97, 104,   3,  18,  38,  96,
        99,  34,  36,  63,  80,  85,   5,  40,  45,  66,  82,  47,  87,
       102, 110,  12,  67,  76,  86,  19,  49,  88, 105,  10,  78, 107,
         2,  20,  29,  33,  71,  17,  89,  94,  44, 103, 106, 114])

In [ ]:
disease_concat_df = pd.concat([disease_df,disease_scaled_df],axis=1)
disease_concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Disease               349 non-null    int64  
 1   Fever                 349 non-null    int64  
 2   Cough                 349 non-null    int64  
 3   Fatigue               349 non-null    int64  
 4   Difficulty Breathing  349 non-null    int64  
 5   Age                   349 non-null    int64  
 6   Gender                349 non-null    int64  
 7   Blood Pressure        349 non-null    int64  
 8   Cholesterol Level     349 non-null    int64  
 9   Outcome Variable      349 non-null    int64  
 10  Age                   349 non-null    float64
dtypes: float64(1), int64(10)
memory usage: 30.1 KB


In [ ]:
y = disease_concat_df["Outcome Variable"].values
X = disease_concat_df.drop(["Outcome Variable"],1).values
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

<ipython-input-11-27d80cc15967>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = disease_concat_df.drop(["Outcome Variable"],1).values


In [ ]:
disease_scaler = StandardScaler()
X_scaler = disease_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
len(X_train)
# X_train_shaped = X_train_scaled.reshape()

261

In [ ]:
disease_input_features = len(X_train[0])
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 2

disease_nn = tf.keras.models.Sequential()

# disease_nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=disease_input_features, activation="ReLU")
# )

disease_nn.add(
    tf.keras.layers.InputLayer(input_shape=(10))
)

disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="ReLU"))

# Second hidden layer
disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# # Third layer
# disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# # Fourth layer
# disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

#Output layer
disease_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
disease_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                550       
                                                                 
 dense_1 (Dense)             (None, 30)                1530      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 2,111
Trainable params: 2,111
Non-trainable params: 0
_________________________________________________________________


In [ ]:
disease_nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["categorical_accuracy","accuracy"])

In [ ]:
print(y_train)

[0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0
 0 0 1 0 1 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0
 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 0 0
 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1
 0 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 0
 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0
 1 1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 1
 1 0]


In [ ]:
disease_fit_model = disease_nn.fit(X_train_scaled,y_train,epochs=30)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


9/9 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 2/30
9/9 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 3/30
9/9 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 4/30
9/9 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 5/30
9/9 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 6/30
9/9 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 7/30
9/9 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5211
Epoch 8/30
9/9 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - categorical

In [ ]:
model_loss, model_accuracy = disease_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.5682 - 185ms/epoch - 62ms/step


ValueError: ignored

In [ ]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.InputLayer(input_shape=(17)))
  hidden_nodes_layer1 =  16
  hidden_nodes_layer2 = 8
  hidden_nodes_layer3 = 4
  hidden_nodes_layer4 = 2
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='ReLU'))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer1))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer2))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer3))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer4))
  model.add(tf.keras.layers.Dense(1,activation="softmax"))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
project_tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
project_tuner.search(X_train_scaled, y_train, epochs=25, validation_split=0.2)